In [1]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from nba_api.stats.endpoints import playernextngames


In [2]:
from sklearn.ensemble import RandomForestRegressor  
from sklearn.metrics import mean_squared_error
import numpy as np

In [3]:
from datetime import datetime
from datetime import date

today = date.today()
print (today)
def convert_date_format(today):

    # Format the date and convert the month to uppercase
    new_format = today.strftime('%b %d, %Y').upper()

    return new_format

# Example usage
converted_today = convert_date_format(today)
print(converted_today)

2024-01-10
JAN 10, 2024


In [4]:
players_unfiltered = players.get_active_players()
all_players = []
no_minute_players = [203648, 1627853, 1631116, 1641777, 1641931, 1641778, 1630547, 1629718, 1631219, 1628427, 1630608, 1631321, 1641740, 1641730, 1641741,
1631120, 1631217, 1628981, 1630259, 1631241, 1641788, 1629232, 1631221, 1631367, 1631243, 1630284, 1641707, 1630207, 1641793, 1627745, 1630548, 1630233, 1631132, 1627788, 1630639,1630572, 1641754,  
1630572, 1631121, 1641757,1631169, 1631111,1630227,1641936, 1641763, 1630695,1630563, 1641871, 1631197, 1641811, 1641712, 1631257, 1641766, 1630611, 1629678, 1641729, 1641767, 1630696, 1631173, 1631306, 1630678, 1631210, 1641816,1630649
,1630688,1630528, 1641775, 1630322, 1629623,1631246, 1630592, 1631209, 1631466, 1630606, 1641771, 201937, 1630296, 1629610, 1629660, 201980, 1631260, 1641926, 1630535, 1631112, 1641735, 1630527, 1631230, 1628385, 1630264, 1628962
,1641721, 1627814, 1629717, 1630658, 1630586, 1630568, 1630215, 1630235, 1631199, 1630604, 1631254, 201988, 201988, 1630647, 1629618, 1631115, 1630613, 1630625, 1628964, 1641723, 1631165, 1628984, 203925, 1641713, 1641749, 1641753, 1626246, 1630231, 1629726, 1631303, 1630600, 1629004, 1641806, 1628373, 1630196
                    ,1630243, 1631157, 1630846, 1631102, 1631104, 1641727, 203469, 1631220, 1629677, 1631311, 1631171, 1630184, 203500, 1629716, 1630181] 
#filter out players who get few minutes to save time
print(len(no_minute_players))
for i in players_unfiltered:
    if i['id'] not in  no_minute_players:
        all_players.append(i)
    else:
        print(i['full_name'])
    
print(all_players)


139
Steven Adams
Thanasis Antetokounmpo
Ryan Arcidiacono
Udoka Azubuike
Amari Bailey
Patrick Baldwin Jr.
Mo Bamba
Dalano Banton
Dominick Barlow
Charles Bediako
Onuralp Bitim
Leaky Black
Marques Bolden
Brandon Boston Jr.
James Bouknight
Armoni Brooks
Kendall Brown
Greg Brown III
Charlie Brown Jr.
Kobe Bufkin
Jared Butler
John Butler Jr.
Vlatko Cancar
Colin Castleton
Malcolm Cazalon
Josh Christopher
Sidy Cissoko
Jaylen Clark
Noah Clowney
Ricky Council IV
JD Davison
Dexter Dennis
Moussa Diabate
Keon Ellis
Bruno Fernando
Jordan Ford
Trent Forrest
Javon Freeman-Liberty
Alex Fudge
Usman Garuba
Luka Garza
Kaiser Gates
Harry Giles III
Anthony Gill
Collin Gillespie
Jacob Gilyard
Devonte' Graham
AJ Green
Danny Green
Mouhamed Gueye
Ron Harper Jr.
Joe Harris
Kevon Harris
Taylor Hendricks
Nate Hinton
D'Moi Hodge
GG Jackson
DaQuan Jeffries
Ty Jerome
Keyontae Johnson
Damian Jones
Johnny Juzang
Braxton Key
Nathan Knight
Christian Koloko
Furkan Korkmaz
A.J. Lawson
Damion Lee
Justin Lewis
Maxwell Lewis


In [5]:

#initialize empty lists that will put player id into
player_ids = []
for i in all_players:
    player_ids.append(i['id']) 
print (player_ids)

[1630173, 1628389, 1630534, 1630583, 1629638, 1628960, 1628386, 1630631, 203937, 203507, 1630175, 1628384, 1630166, 1629028, 1628963, 1630163, 1628366, 1631094, 1630217, 203084, 1630567, 1629628, 1629646, 1641734, 1628966, 201587, 203078, 1627736, 1630699, 202722, 201976, 1630180, 1629048, 202687, 1641710, 203992, 202711, 1629626, 1626164, 1628449, 1631103, 1631128, 1628969, 1628970, 1629052, 1627763, 1628415, 1628971, 1627759, 1641738, 1629650, 1628972, 1628973, 1628418, 203493, 202692, 202710, 1631288, 203484, 1641739, 203991, 1628975, 1628976, 1627936, 1630577, 1631108, 1629634, 203903, 1629651, 1629599, 1628381, 1628380, 201144, 1626192, 1641731, 203496, 1628470, 203109, 1630595, 203552, 201939, 1630700, 203076, 1631098, 201942, 1628978, 1641711, 1631172, 203915, 1629029, 1629652, 1630245, 203083, 1630537, 1627739, 201142, 1631105, 1631106, 1630162, 1630556, 203954, 1629234, 203957, 1627827, 1630201, 1631323, 203095, 1628368, 1628365, 1629655, 201568, 1629636, 1641718, 202331, 1630

In [6]:
playergamelogs = []
today_players = []

for i in player_ids:
    playergamelogs.append(playergamelog.PlayerGameLog(player_id= i))
    print(i)
    today = playernextngames.PlayerNextNGames(player_id= i)
    player = today.get_data_frames()[0]
    
    df = pd.DataFrame(player)
    
    if(df.iloc[0, df.columns.get_loc('GAME_DATE')] == converted_today):
        today_players.append(i)


print(today_players)
    

1630173
1628389
1630534
1630583
1629638
1628960
1628386
1630631
203937
203507
1630175
1628384
1630166
1629028
1628963
1630163
1628366
1631094
1630217
203084
1630567
1629628
1629646
1641734
1628966
201587
203078
1627736
1630699
202722
201976
1630180
1629048
202687
1641710
203992
202711
1629626
1626164
1628449
1631103
1631128
1628969
1628970
1629052
1627763
1628415
1628971
1627759
1641738
1629650
1628972
1628973
1628418
203493
202692
202710
1631288
203484
1641739
203991
1628975
1628976
1627936
1630577
1631108
1629634
203903
1629651
1629599
1628381
1628380
201144
1626192
1641731
203496
1628470
203109
1630595
203552
201939
1630700
203076
1631098
201942
1628978
1641711
1631172
203915
1629029
1629652
1630245
203083
1630537
1627739
201142
1631105
1631106
1630162
1630556
203954
1629234
203957
1627827
1630201
1631323
203095
1628368
1628365
1629655
201568
1629636
1641718
202331
1630581
1628983
203497
1630692
203932
201569
203924
203110
1630224
201145
1630182
1631100
1629656
1629060
1630169
16301

In [7]:
playergamelogs[14].get_data_frames()[0] #example getting a player

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,1628963,0022300453,"JAN 01, 2024",DET @ HOU,L,9,1,2,0.500,...,0,0,2,1,0,0,0,2,1,1
1,22023,1628963,0022300394,"DEC 23, 2023",DET @ BKN,L,15,2,4,0.500,...,3,5,0,0,0,1,0,5,-9,1
2,22023,1628963,0022300374,"DEC 21, 2023",DET vs. UTA,L,31,10,12,0.833,...,3,5,0,0,0,0,3,22,6,1
3,22023,1628963,0022300352,"DEC 18, 2023",DET @ ATL,L,23,7,12,0.583,...,0,1,0,0,0,2,2,17,-10,1
4,22023,1628963,0022300334,"DEC 16, 2023",DET @ MIL,L,22,6,11,0.545,...,4,7,1,1,2,0,3,12,-17,1
5,22023,1628963,0022301217,"DEC 08, 2023",DET @ ORL,L,23,4,7,0.571,...,3,5,0,0,0,4,1,9,-19,1
6,22023,1628963,0022300283,"DEC 02, 2023",DET vs. CLE,L,7,0,1,0.000,...,2,3,0,1,0,0,1,0,-11,1
7,22023,1628963,0022300268,"NOV 30, 2023",DET @ NYK,L,14,3,6,0.500,...,0,3,1,0,1,1,1,8,-7,1
8,22023,1628963,0022300258,"NOV 29, 2023",DET vs. LAL,L,14,2,4,0.500,...,1,2,0,0,0,0,0,4,-4,1
9,22023,1628963,0022300047,"NOV 24, 2023",DET @ IND,L,14,3,6,0.500,...,3,6,2,0,3,1,1,6,-8,1


In [8]:
playergamelogs

In [9]:
logs_dict = {}
today_logs_dict = {}



for i in range(len(player_ids)):
    player = playergamelogs[i].get_data_frames()[0]
    df = pd.DataFrame(player)
    logs_dict[all_players[i]['full_name']] = df
    if(player_ids[i] in today_players):
        today_logs_dict[all_players[i]['full_name']] = df
        

        


In [10]:
#print(logs_dict)
print(today_players)
print(today_logs_dict)

[1628389, 1630534, 1629638, 1630631, 203937, 1630166, 1628963, 1630163, 1628366, 203084, 1630567, 1629628, 1629646, 201587, 202722, 201976, 1630180, 203992, 202711, 1628449, 1631103, 1631128, 1628970, 1629052, 1628415, 1628971, 1627759, 1641738, 1628972, 1628418, 203493, 202692, 202710, 1631288, 203484, 203991, 1628975, 1627936, 1630577, 203903, 1629599, 1628381, 1628380, 201144, 1641731, 203496, 1628470, 1630595, 201939, 1630700, 1631098, 201942, 1641711, 1631172, 1629652, 1630245, 203083, 1630537, 1627739, 1631105, 1631106, 1630162, 1630556, 203954, 1631323, 1628368, 1629655, 201568, 1641718, 202331, 1630581, 1628983, 203497, 203932, 203110, 1630224, 201145, 1631100, 1630169, 1630181, 201935, 202699, 1630573, 1641722, 1630165, 202330, 1629639, 1627741, 1629312, 1628988, 201950, 203200, 1631096, 201143, 1629659, 1627863, 1628989, 1630643, 1629631, 1630538, 1627742, 1631093, 1630543, 202704, 1631218, 1631170, 1630198, 1630552, 1629640, 203999, 1641732, 1630529, 1630200, 1626145, 201599

In [11]:
for key in logs_dict.keys():
    logs_dict[key] = logs_dict[key].iloc[::-1]

In [12]:
print(logs_dict)

{'Precious Achiuwa':    SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
29     22023    1630173  0022300069  OCT 25, 2023  TOR vs. MIN  W   24    4   
28     22023    1630173  0022300082  OCT 27, 2023    TOR @ CHI  L   23    2   
27     22023    1630173  0022300092  OCT 28, 2023  TOR vs. PHI  L   18    3   
26     22023    1630173  0022300174  NOV 11, 2023    TOR @ BOS  L   19    1   
25     22023    1630173  0022300189  NOV 13, 2023  TOR vs. WAS  W   19    5   
24     22023    1630173  0022300195  NOV 15, 2023  TOR vs. MIL  L   17    4   
23     22023    1630173  0022300031  NOV 17, 2023  TOR vs. BOS  L   24    2   
22     22023    1630173  0022300211  NOV 19, 2023  TOR vs. DET  W   19    5   
21     22023    1630173  0022300038  NOV 21, 2023    TOR @ ORL  L   15    2   
20     22023    1630173  0022300046  NOV 24, 2023  TOR vs. CHI  W   18    3   
19     22023    1630173  0022300251  NOV 26, 2023    TOR @ CLE  L   13    1   
18     22023    1630173  002230

In [15]:
pts_model_dict = {}
for key in logs_dict.keys():
    print(key)
    features = ['PTS'] #'REB', 'AST', 'STL', 'BLK']  # Features of interest, multiple stats still a work in progress
    df = logs_dict[key]
    X = []
    y = []
    if(df.shape[0] < 12):
        continue
    for feature in features:
        window_size = 10
        for start in range(len(df) - window_size):
            window = df[feature].iloc[start:start + window_size]  #find the average of 10 consecutive games
            X.append(window.mean()) #the average of the 10 games is one element of the training X set
            y.append(df[feature].iloc[start+ window_size]) #find the game after the chosen 10 games, the corresponding y element of the training set
            
            
    
        
    X = np.array(X)
    y = np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = X_train.reshape(-1,1)
    X_test = X_test.reshape(-1,1)
    
    
    
    # Training the model
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluating the model
    predictions = model.predict(X_test)
    # print(predictions)
    pts_model_dict[key] = [model]
 
    mse = mean_squared_error(y_test, predictions)
    pts_model_dict[key].append(mse)
    print(f"Mean Squared Error: {np.sqrt(mse)}")

    recent_games = df[feature].iloc[-window_size:]
    next_game_prediction = model.predict(np.array([recent_games.mean()]).reshape(1, -1))
    recent_games_avg = recent_games.mean()
    print(f"Next Game Prediction: {next_game_prediction[0]}")
    print("Based on Avg prev 10: " + str(recent_games_avg))
   
    

    
    # print(X)
    # print(y)


Precious Achiuwa
Mean Squared Error: 7.405899675258908
Next Game Prediction: 2.86
Based on Avg prev 10: 4.1
Bam Adebayo
Mean Squared Error: 10.11545005726938
Next Game Prediction: 19.206500000000002
Based on Avg prev 10: 21.5
Ochai Agbaji
Mean Squared Error: 3.16318207400923
Next Game Prediction: 3.055
Based on Avg prev 10: 5.1
Santi Aldama
Mean Squared Error: 5.229316465317858
Next Game Prediction: 8.363999999999999
Based on Avg prev 10: 7.1
Nickeil Alexander-Walker
Mean Squared Error: 2.5434749933244216
Next Game Prediction: 8.38
Based on Avg prev 10: 4.5
Grayson Allen
Mean Squared Error: 9.5147704328002
Next Game Prediction: 12.62
Based on Avg prev 10: 16.2
Jarrett Allen
Mean Squared Error: 5.33860593686151
Next Game Prediction: 24.54
Based on Avg prev 10: 18.8
Jose Alvarado
Mean Squared Error: 5.492073045999351
Next Game Prediction: 7.215
Based on Avg prev 10: 7.0
Kyle Anderson
Mean Squared Error: 2.8863720612433053
Next Game Prediction: 2.926
Based on Avg prev 10: 5.8
Giannis Ante

In [16]:
pts_td_model_dict = {}
for key in today_logs_dict.keys():
    print(key)
    features = ['PTS'] #'REB', 'AST', 'STL', 'BLK']  # Features of interest, multiple stats still a work in progress
    df = logs_dict[key]
    X = []
    y = []
    if(df.shape[0] < 12):
        continue
    for feature in features:
        window_size = 10
        for start in range(len(df) - window_size):
            window = df[feature].iloc[start:start + window_size]  #find the average of 10 consecutive games
            X.append(window.mean()) #the average of the 10 games is one element of the training X set
            y.append(df[feature].iloc[start+ window_size]) #find the game after the chosen 10 games, the corresponding y element of the training set
            
            
    
        
    X = np.array(X)
    y = np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = X_train.reshape(-1,1)
    X_test = X_test.reshape(-1,1)
    
    
    
    # Training the model
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluating the model
    predictions = model.predict(X_test)
    # print(predictions)
    pts_model_dict[key] = [model]
 
    mse = mean_squared_error(y_test, predictions)
    pts_model_dict[key].append(mse)
    print(f"Mean Squared Error: {np.sqrt(mse)}")

    recent_games = df[feature].iloc[-window_size:]
    next_game_prediction = model.predict(np.array([recent_games.mean()]).reshape(1, -1))
    print(f"Next Game Prediction: {next_game_prediction[0]}")
      recent_games_avg = recent_games.mean()
    print(f"Next Game Prediction: {next_game_prediction[0]}")
    print("Based on Avg prev 10: " + str(recent_games_avg))
    
    # print(X)
    # print(y)





    
    


IndentationError: unexpected indent (4259317487.py, line 45)